# SQL-запросы (PostgreSQL)

> В этом проекте проводится работа с базой данных [StackOverflow](https://stackoverflow.com/) — сервиса вопросов и ответов о программировании. В исследуемой базе данных хранятся данные о постах за 2008 год, а также информация об оценках, которые эти посты получили.   
В данной работе реализовано двадцать SQL-запросов с применением различных оконных функций.

## ER-диаграмма

![ER-диаграмма](https://pictures.s3.yandex.net/resources/Frame_353_1_1664969443.png)

## Описание таблиц в базе данных

### `stackoverflow.badges`
Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про *PostgreSQL*, может получить значок *postgresql*. 
* `id` — идентификатор значка, первичный ключ таблицы;
* `name` — название значка;
* `user_id` — идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице `users`;
* `creation_date` — дата присвоения значка.

### `stackoverflow.post_types`
Содержит информацию о типе постов.
* `id` — идентификатор поста, первичный ключ таблицы;
* `type` — тип поста:
  * `Question` — пост с вопросом;
  * `Answer` — пост с ответом.
  
### `stackoverflow.posts`
Содержит информацию о постах.
* `id` — идентификатор поста, первичный ключ таблицы;
* `title` — заголовок поста;
* `creation_date` — дата создания поста;
* `favorites_count` — число, которое показывает, сколько раз пост добавили в «Закладки»;
* `last_activity_date` — дата последнего действия в посте, например комментария;
* `last_edit_date` — дата последнего изменения поста;
* `user_id` — идентификатор пользователя, который создал пост, внешний ключ к таблице `users`;
* `parent_id` — если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом;
* `post_type_id` — идентификатор типа поста, внешний ключ к таблице `post_types`;
* `score` — количество очков, которое набрал пост;
* `views_count` — количество просмотров.

### `stackoverflow.users`
Содержит информацию о пользователях.
* `id` — идентификатор пользователя, первичный ключ таблицы;
* `creation_date` — дата регистрации пользователя;
* `display_name` — имя пользователя;
* `last_access_date` — дата последнего входа;
* `location` — местоположение;
* `reputation` — очки репутации, которые получают за хорошие вопросы и полезные ответы;
* `views` — число просмотров профиля пользователя.

### `stackoverflow.vote_types`
Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту.
* `id` — идентификатор типа голоса, первичный ключ;
* `name` — название метки:
  * `UpMod` — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными;
  * `DownMod` — такую отметку получают посты, которые показались пользователям наименее полезными;
  * `Close` — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы;
  * `Offensive` — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста;
  * `Spam` — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.
  
### `stackoverflow.votes`
Содержит информацию о голосах за посты.
* `id` — идентификатор голоса, первичный ключ;
* `post_id` — идентификатор поста, внешний ключ к таблице `posts`;
* `user_id` — идентификатор пользователя, который поставил посту голос, внешний ключ к таблице `users`;
* `bounty_amount` — сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту;
* `vote_type_id` — идентификатор типа голоса, внешний ключ к таблице `vote_types`;
* `creation_date` — дата назначения голоса.

# Задания

**Задача 1**  
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

```sql
SELECT COUNT(p.id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
WHERE pt.type = 'Question'
  AND (score > 300
       OR favorites_count >= 100);
```

---

**Задача 2**  
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

```sql
SELECT ROUND(AVG(question_cnt))
FROM
  (SELECT creation_date::date AS question_date,
          COUNT(p.id) AS question_cnt
   FROM stackoverflow.posts AS p
   JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
   WHERE pt.type = 'Question'
     AND creation_date::date BETWEEN '2008-11-01' AND '2008-11-18'
   GROUP BY creation_date::date) AS Question;
```

---

**Задача 3**  
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

```sql
SELECT COUNT(DISTINCT u.id) AS USER
FROM stackoverflow.users AS u
JOIN stackoverflow.badges AS b ON u.id = b.user_id
WHERE u.creation_date::date = b.creation_date::date;
```

---

**Задача 4**  
Сколько уникальных постов пользователя с именем *Joel Coehoorn* получили хотя бы один голос?

```sql
WITH votes_cnts AS
  (SELECT p.id AS id_post,
          COUNT(v.id) AS votes_cnt
   FROM stackoverflow.users AS u
   JOIN stackoverflow.posts AS p ON p.user_id = u.id
   JOIN stackoverflow.votes AS v ON v.post_id = p.id
   WHERE display_name = 'Joel Coehoorn'
   GROUP BY p.id)
SELECT COUNT(id_post)
FROM votes_cnts
WHERE votes_cnt > 0;
```

---

**Задача 5**  
Выгрузите все поля таблицы `vote_types`. Добавьте к таблице поле `rank`, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю `id`.

```sql
SELECT *,
       RANK() OVER (
                    ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id;
```

---

**Задача 6**  
Отберите 10 пользователей, которые поставили больше всего голосов типа `Close`. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

```sql
SELECT v.user_id AS user_votes,
       COUNT (v.id) AS votes_cnt
FROM stackoverflow.votes AS v
JOIN stackoverflow.vote_types AS vt ON vt.id = v.vote_type_id
WHERE name = 'Close'
GROUP BY v.user_id
ORDER BY votes_cnt DESC,
         v.user_id DESC
LIMIT 10;
```

---

**Задача 7**  
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.  
Отобразите несколько полей:
* идентификатор пользователя;
* число значков;
* место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.

Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

```sql
SELECT user_id,
       COUNT (id) AS badges_cnt,
             DENSE_RANK() OVER (
                                ORDER BY COUNT (id) DESC)
FROM stackoverflow.badges
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY badges_cnt DESC,
         user_id
LIMIT 10;
```

---

**Задача 8**  
Сколько в среднем очков получает пост каждого пользователя?  
Сформируйте таблицу из следующих полей:
* заголовок поста;
* идентификатор пользователя;
* число очков поста;
* среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

```sql
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE title IS NOT NULL
  AND score != 0;
```

---

**Задача 9**  
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

```sql
SELECT title
FROM stackoverflow.posts
WHERE user_id IN
    (SELECT user_id
     FROM stackoverflow.badges
     GROUP BY user_id
     HAVING COUNT(id) > 1000)
  AND title IS NOT NULL;
```

---

**Задача 10**  
Напишите запрос, который выгрузит данные о пользователях из США (англ. *United States*). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
* пользователям с числом просмотров больше либо равным 350 присвойте группу `1`;
* пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу `2`;
* пользователям с числом просмотров меньше 100 — группу `3`.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

```sql
SELECT id,
       VIEWS,
       CASE
           WHEN VIEWS >= 350 THEN 1
           WHEN VIEWS >= 100
                AND VIEWS < 350 THEN 2
           ELSE 3
       END AS GROUP
FROM stackoverflow.users
WHERE LOCATION LIKE '%United States%'
  AND VIEWS != 0
ORDER BY VIEWS DESC;
```

---

**Задача 11**  
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

```sql
WITH us_users AS
  (SELECT id AS user_id,
          VIEWS AS views_cnt,
                   CASE
                       WHEN VIEWS >= 350 THEN 1
                       WHEN VIEWS >= 100
                            AND VIEWS < 350 THEN 2
                       ELSE 3
                   END AS groups
   FROM stackoverflow.users
   WHERE LOCATION LIKE '%United States%'
     AND VIEWS != 0)
SELECT user_id,
       groups,
       views_cnt
FROM
  (SELECT user_id,
          views_cnt,
          groups,
          MAX(views_cnt) OVER (PARTITION BY groups
                               ORDER BY views_cnt DESC) AS max_views
   FROM us_users) AS max_us
WHERE views_cnt = max_views
ORDER BY views_cnt DESC,
         user_id;
```

---

**Задача 12**  
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
* номер дня;
* число пользователей, зарегистрированных в этот день;
* сумму пользователей с накоплением.

```sql
SELECT days,
       users_cnt,
       SUM(users_cnt) OVER (
                            ORDER BY days)
FROM
  (SELECT EXTRACT (DAY
                   FROM creation_date) AS days,
                  COUNT(id) AS users_cnt
   FROM stackoverflow.users
   WHERE creation_date :: date BETWEEN '2008-11-01' AND '2008-11-30'
   GROUP BY EXTRACT (DAY
                     FROM creation_date)) AS user_november;
```

---

**Задача 13**  
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
* идентификатор пользователя;
* разницу во времени между регистрацией и первым постом.

```sql
SELECT DISTINCT p.user_id,
                MIN(p.creation_date) OVER (PARTITION BY p.user_id) - u.creation_date AS interval
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id;
```

---

**Задача 14**  
Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

```sql
SELECT DATE_TRUNC('month', creation_date)::date AS month_posts,
       SUM(views_count) AS views_count
FROM stackoverflow.posts
WHERE EXTRACT (YEAR
               FROM creation_date) = 2008
GROUP BY month_posts
ORDER BY views_count DESC;
```

---

**Задача 15**  
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений `user_id`. Отсортируйте результат по полю с именами в лексикографическом порядке.

```sql
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
WHERE p.creation_date::date BETWEEN u.creation_date::date AND u.creation_date::date + INTERVAL '1 month'
  AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(DISTINCT p.id) > 100;
```

---

**Задача 16**  
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

```sql
SELECT DATE_TRUNC('month', p.creation_date)::date AS month_posts,
       COUNT (DISTINCT id) AS user_count
FROM stackoverflow.posts AS p
WHERE user_id IN
    (SELECT DISTINCT u.id
     FROM stackoverflow.users AS u
     JOIN stackoverflow.posts AS p ON p.user_id = u.id
     WHERE u.creation_date :: date BETWEEN '2008-09-01' AND '2008-09-30'
       AND p.creation_date :: date BETWEEN '2008-12-01' AND '2008-12-31' )
  AND p.creation_date::date BETWEEN '2008-01-01' AND '2008-12-31'
GROUP BY month_posts
ORDER BY month_posts DESC;
```

---

**Задача 17**  
Используя данные о постах, выведите несколько полей:
* идентификатор пользователя, который написал пост;
* дата создания поста;
* количество просмотров у текущего поста;
* сумму просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

```sql
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id
                              ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id,
         creation_date;
```

---

**Задача 18**  
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

```sql
SELECT ROUND(AVG (active_days))
FROM
  (SELECT user_id,
          COUNT(DISTINCT creation_date::date) AS active_days
   FROM stackoverflow.posts
   WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
   GROUP BY user_id) AS cnt;
```

---

**Задача 19**  
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
* номер месяца;
* количество постов за месяц;
* процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

```sql
WITH posts_month AS
  (SELECT EXTRACT (MONTH
                   FROM creation_date) AS month_number,
                  COUNT(id) AS posts_cnt
   FROM stackoverflow.posts
   WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
   GROUP BY EXTRACT (MONTH
                     FROM creation_date)
   ORDER BY month_number)
SELECT *,
       ROUND((posts_cnt - LAG(posts_cnt) OVER ()) * 100 / LAG(posts_cnt) OVER ()::numeric, 2)
FROM posts_month;
```

---

**Задача 20**  
Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
* номер недели;
* дата и время последнего поста, опубликованного на этой неделе.

```sql
WITH WEEK AS
  (SELECT EXTRACT(WEEK
                  FROM creation_date) AS week_number,
          MAX(creation_date) OVER (
                                   ORDER BY EXTRACT(WEEK
                                                    FROM creation_date))
   FROM stackoverflow.posts
   WHERE user_id =
       (SELECT user_id
        FROM stackoverflow.posts
        GROUP BY user_id
        ORDER BY COUNT(id) DESC
        LIMIT 1)
     AND creation_date :: date BETWEEN '2008-10-01' AND '2008-10-31'
   ORDER BY creation_date)
SELECT DISTINCT *
FROM WEEK
ORDER BY week_number;
```

---